In [ ]:
import sqlite3
from sqlite3 import Error

connection = sqlite3.connect("atendimento_nae.sqlite")
## Função de alteração (insert, update, delete...)
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print(f"Query {query.split('(')[0].strip()} executada.")
        if cursor.rowcount != -1: print(f"{cursor.rowcount} linha(s) afetadas")

    except Error as e:
        print(f"Erro: '{e}'")


# Função de leitura de dados (select)
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()

        return result
    except Error as e:
        print(f"Erro: '{e}'")

        connection = sqlite3.connect("atendimento_nae.sqlite")

query = """
create table Profissional(
	nome varchar (100),
  CPF varchar(14) primary key,
  email varchar(50),
  telefone varchar(15),
  dataNasc date
);
"""
execute_query(connection, query)

query = """
create table Aluno(
	nome varchar(100),
  RA char(7) primary key,
  telefone varchar(15),
  dataNasc date
);
"""
execute_query(connection, query)

query = """
create table Responsavel(
	nome varchar(100),
  cpf varchar(14) primary key,
  telefone varchar(15),
  grauParentesco text
);
"""
execute_query(connection, query)

query = """
create table Prontuario(
	ID integer primary key,
	dataProntuario date,
  descricao text,
  condicaoRestricao text
);
"""
execute_query(connection, query)

### Esse aqui precisa arrumar
##### A chave primária de anexo é o ID do prontuário? Só pode ter um anexo por prontuário?
# query = """
# create table Anexo(
# 	IDProntuario integer primary key,
#   arquivo text,
#   constraint fk_Anexo foreign key (IDProntuario) references Prontuario(ID)
# );
# """

query = """
create table fazProntuario(
	CPFProfissional varchar(14),
  RAaluno char(7),
  IDprontuario int,

  primary key (CPFProfissional, RAAluno, Idprontuario),
  foreign key (CPFProfissional) references Profissional(CPF) on delete restrict,
  foreign key (RAaluno) references Aluno(RA) on delete restrict,
  foreign key (IDprontuario) references Prontuario(ID) on delete restrict
);
"""
execute_query(connection, query)

query = """
create table alunoPossuiResponsavel(
	RAAluno char(7),
  cpfResponsavel varchar(14),

  primary key (RAAluno, cpfResponsavel),
  foreign key (RAAluno) references Aluno(RA) on delete restrict,
  foreign key (cpfResponsavel) references Responsavel(cpf) on delete restrict
);
"""
execute_query(connection, query)

query = """
insert into Aluno (nome, RA, telefone, dataNasc) values
('Rafael Felipe', 'A123456', '(11)91234-5678', '2006-03-15'),
('Paulo Gabriel', 'A123457', '(11)92345-6789', '2005-08-22'),
('Pedro Martins', 'A123458', '(11)93456-7890', '2006-01-10'),
('Raul Miguel', 'A123459', '(11)94567-8901', '2005-12-05'),
('Arthur Pacheco', 'A123460', '(11)95678-9012', '2006-07-19'),
('Heros Pacheco', 'A123461', '(11)96789-0123', '2005-10-30'),
('Daniel Borges', 'A123462', '(11)97890-1234', '2006-04-25'),
('Yan Gabriel', 'A123463', '(11)98901-2345', '2005-09-14');
"""
execute_query(connection, query)

query = """
insert into Profissional (nome, CPF, email, telefone, dataNasc) values
('Thamara', '123.456.789-00', 'thamara@email.com', '(11)91234-5678', '1990-05-12'),
('Listiane', '987.654.321-00', 'listiane@email.com', '(11)99876-5432', '1988-11-23');
"""
execute_query(connection, query)

query = """
insert into Responsavel (nome, cpf, telefone, grauParentesco) values
('Maria Silva', '111.222.333-44', '(11)91234-5678', 'Mãe'),
('João Pereira', '555.666.777-88', '(11)98765-4321', 'Pai'),
('Ana Costa', '999.888.777-66', '(11)99887-6655', 'Tia');
"""
execute_query(connection, query)

query = """
insert into alunoPossuiResponsavel (RAAluno, cpfResponsavel) values
('A123456', '111.222.333-44'),
('A123457', '555.666.777-88'),
('A123458', '999.888.777-66');
"""
execute_query(connection, query)

query = """
insert into Prontuario (ID, dataProntuario, descricao, condicaoRestricao) values
(1, '2025-06-01', 'Consulta inicial com avaliação psicológica', 'Nenhuma restrição'),
(2, '2025-06-10', 'Sessão de acompanhamento para ansiedade', 'Evitar estresse excessivo'),
(3, '2025-06-15', 'Avaliação para TDAH e planejamento terapêutico', 'Restrição ao uso de eletrônicos antes da consulta');
"""
execute_query(connection, query)

query = """
insert into fazProntuario (CPFProfissional, RAaluno, IDprontuario) values
('123.456.789-00', 'A123456', 1),
('987.654.321-00', 'A123457', 2),
('123.456.789-00', 'A123458', 3);
"""
execute_query(connection, query)

Query create table Profissional executada.
Query create table Aluno executada.
Query create table Responsavel executada.
Query create table Prontuario executada.
Query create table fazProntuario executada.
Query create table alunoPossuiResponsavel executada.
Query insert into Aluno executada.
8 linha(s) afetadas
Query insert into Profissional executada.
2 linha(s) afetadas
Query insert into Responsavel executada.
3 linha(s) afetadas
Query insert into alunoPossuiResponsavel executada.
3 linha(s) afetadas
Query insert into Prontuario executada.
3 linha(s) afetadas
Query insert into fazProntuario executada.
3 linha(s) afetadas


ATIVIDADE 01

Objetivo no projeto: caso a profissional deseje saber quantos alunos ela já registrou até o momento


Ação esperada: mostrar a quantidade de alunos que a profissional Thamara, por exemplo, atendeu

In [ ]:
query = """SELECT Profissional.nome, Profissional.CPF, COUNT(fazProntuario.RAAluno) AS total_atendido FROM Profissional JOIN fazProntuario ON Profissional.CPF = fazProntuario.CPFProfissional WHERE Profissional.nome = 'Thamara';"""
execute_query(connection, query)

Objetivo no projeto: Saber quantos alunos de certo nome foram atendidos

Ação esperada: mostrar quantos alunos com o nome "Daniel", por exemplo, foram atendidos

In [ ]:
query = """ SELECT count(*) FROM Aluno WHERE nome = 'Daniel' GROUP BY dataNasc"""
execute_query(connection, query)

ATIVIDADE 02

Objetivo no projeto: caso alguma profissional queira saber quantos alunos ela já registrou

Ação esperada: mostrar a quantidade de alunos que a profissional Listiane, por exemplo, atendeu

In [ ]:
query = """ SELECT p.nome as nome_profissional, count(f.RA) as total_alunos FROM fazProntuario f JOIN Aluno a ON f.RAaluno = a.RA JOIN Profissional p on f.CPFProfissional = p.CPF WHERE nome_profissional = 'Listiane' GROUP BY nome_profissional"""
execute_query(connection, query)

ATIVIDADE 03

Objetivo no projeto: tendo em vista que uma consulta para saber quem é o responsavel por x aluno será muito usada, a view facilitará esse procedimento, evitando de chamar um select toda vez

Ação esperada: mostrar a relação aluno - responsável

In [ ]:
query = """ CREATE VIEW relacao_ar AS SELECT r.nome as nome_responsavel, a.nome as nome_aluno FROM alunoPossuiResponsavel pr JOIN Aluno a ON pr.RAaluno = a.RA JOIN pr.cpfResponsavel = r.CPF GROUP BY nome_aluno """
execute_query(connection, query)

ATIVIDADE 04

Objetivo no projeto: recebendo como parâmetro a chave primária de Profissional, a procedure retornará quantos alunos estão associados ao profissional, facilitando para o usuário do projeto

Ação esperada: mostrar quantos alunos estão associados a determinado profissional

In [ ]:
## procedure
query = """ delimiter $$ CREATE PROCEDURE  ContarAlunosPorProfissional (IN cpf_profissional VARCHAR(14)) BEGIN SELECT COUNT(RAaluno) FROM fazProntuario WHERE CPFProfissional = cpf_profissional; END $$ delimiter ; """
execute_query(connection, query)
query = """ CALL ContarAlunosPorProfissional('123.456.789-00'); """
execute_query(connection, query)

ATIVIDADE 05

Objetivo no projeto: com o objetivo de tornar mais dinâmico, a função exibirá um texto confirmando que uma ação foi feita

Ação esperada: Executará a procedure "ContarAlunosPorProfissional" com base em um parâmetro e exibirá uma mensagem confirmando que a quantidade foi contada

In [ ]:
## function
query = """
delimiter $$ create function RetornarMensagem() returns varchar(100) deterministic
begin
  declare limite varchar(14);
  set limite = '123.456.789-00';
  call ContarAlunosPorProfissional(limite);
  return "O total de alunos atendidos foi retornado.";
end $$
delimiter ;
"""

ATIVIDADE 06

Objetivo no projeto: impedir que um profissional que tenha alunos associados a ele seja excluído

Ação esperada: mostrar uma mensagem de bloqueio ao usuário

In [ ]:

## trigger
query = """
CREATE TRIGGER impedir_exclusao_profissional
BEFORE DELETE ON Profissional
FOR EACH ROW
BEGIN
    SELECT
        CASE
            WHEN EXISTS (
                SELECT 1 FROM fazProntuario WHERE CPFProfissional = OLD.CPF
            )
            THEN RAISE(ABORT, 'Não é possível excluir profissional com atendimentos registrados.')
        END;
END;
"""
execute_query(connection, query)